In [1]:
import numpy as np
import pandas as pd
import smtplib
import torch
import torch.nn as nn
import models

from email.message import EmailMessage
from matplotlib.pylab import plt
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, Dataset
from tqdm.auto import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3, 1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3, 1), 'GB')

Using device: cuda
Quadro RTX 6000
Memory Usage:
Allocated: 0.0 GB
Cached:    0.0 GB


In [21]:
base_dir = '/home/mma6789/Stuff/Studies/sem3/ms_project' #@param {type: 'string'}

dataset = 'US-101' #@param ['I-80', 'US-101']
t_o = 5000 #@param [3000, 4000, 5000] -> observation horizon
t_p = 3500 #@param [1500, 2000, 2500] -> prediction horizon

method = 'LSTMHMM' #@param ['LSTM', 'HMM', 'LSTMHMM']

timesteps = t_o // 100
variables = 5
learning_rate = 0.0002

In [22]:
## Load data splits
test = np.load(f'{base_dir}/data/processed/{dataset}/{t_o}_{t_p}_test.npy')

test_X, test_y = np.split(test, [-3], axis=1)

temp = np.empty((len(test_X), timesteps, variables))
for i in range(len(test_X)):
    temp[i] = np.array(np.split(test_X[i], timesteps))
test_X = temp

In [23]:
test_X = torch.tensor(test_X).float()
test_y = torch.tensor(test_y).float()

In [24]:
class LSTMDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]

test_dataset = LSTMDataset(test_X, test_y)

In [25]:
batch_size = 192

test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [26]:
def test_epoch():
    model.train(False)
    running_loss = 0.0
    y_true = []
    y_pred = []
    
    for batch_index, batch in enumerate(tqdm(test_loader, desc=f'Test')):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        with torch.no_grad():
            output = model(x_batch)
            
        y_true += torch.argmax(y_batch, dim=1).flatten().tolist()
        y_pred += torch.argmax(output, dim=1).cpu().detach().numpy().tolist()
            
    acc = accuracy_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    
    matrix = confusion_matrix(y_true, y_pred)
    class_acc = matrix.diagonal() / matrix.sum(axis=1)
    
    test_acc.append(acc)
    test_f1.append(f1)
    test_class_acc.append(class_acc)
    
    print('Test results:')
    print(f'Acc: {acc}')
    print(f'F1: {f1}')
    print(f'Class Acc: {class_acc}')
    print('***************************************************')

In [27]:
## Load model from file
model = torch.load(f'{base_dir}/models/{method}/{dataset}/model_{t_o}_{t_p}_{learning_rate}.pt')
model.to(device)

Three_LSTMHMM(
  (lcl_model): LSTMHMM(
    (transition_model): TransitionModel()
    (emission_model): LSTMEmissionModel(
      (lstm): LSTM(5, 150, batch_first=True)
      (linear): Linear(in_features=150, out_features=30, bias=True)
    )
  )
  (lk_model): LSTMHMM(
    (transition_model): TransitionModel()
    (emission_model): LSTMEmissionModel(
      (lstm): LSTM(5, 150, batch_first=True)
      (linear): Linear(in_features=150, out_features=30, bias=True)
    )
  )
  (lcr_model): LSTMHMM(
    (transition_model): TransitionModel()
    (emission_model): LSTMEmissionModel(
      (lstm): LSTM(5, 150, batch_first=True)
      (linear): Linear(in_features=150, out_features=30, bias=True)
    )
  )
)

In [28]:
## Test model
test_acc = []
test_f1 = []
test_class_acc = []

test_epoch()

Test:   0%|          | 0/1949 [00:00<?, ?it/s]

Test results:
Acc: 0.9784757725497951
F1: 0.980436711033607
Class Acc: [0.96936456 0.97874857 0.97747587]
***************************************************


In [29]:
## Write metrics to file
metrics = pd.DataFrame(data={
    'test_acc': test_acc, 'test_f1': test_f1, 'test_class_acc': test_class_acc,
})

metrics.to_json(f'{base_dir}/metrics/{method}/{dataset}/testing_{t_o}_{t_p}_{learning_rate}.json')